## Training Example

#### Training a model is very simple, follow this example to train your own model

In [ ]:
#First import the training tool and the torchio library
import sys
sys.path.append('./Radiology_and_AI')
from training.run_training import run_training
import torchio as tio

In [4]:
#Next define what transforms you want applied to the training data
#Both the training and validation data must have the same normalization and data preparation steps
#Only the training samples should have the augmentations applied
#Any transforms found at https://torchio.readthedocs.io/transforms/transforms.html can be applied
#Keep track of the  normalization and data preparation steps steps performed, you will need to apply the to all data passed into the model into the future

#These transforms are applied to data before it is used for training the model
training_transform = tio.Compose([
    #Normalization
    tio.ZNormalization(masking_method=tio.ZNormalization.mean), 
    
    #Augmentation
    #Play around with different augmentations as you desire, refer to the torchio docs to see how they work
    tio.RandomNoise(p=0.5),
    tio.RandomGamma(log_gamma=(-0.3, 0.3)),
    tio.RandomElasticDeformation(),
    
    #Preparation
    tio.CropOrPad((240, 240, 160)), #Crop/pad the images to a dimension your model can handle, our default unnet model requires the dimensions be multiples of 8
    tio.OneHot(num_classes=5), #Set num_classes to the max segmentation label + 1
    
])

#These transforms are applied to data before it is used to determined the performance of the model on the validation set
validation_transform = tio.Compose([
    #Normalization
    tio.ZNormalization(masking_method=tio.ZNormalization.mean),
    
    #Preparation
    tio.CropOrPad((240, 240, 160)),        
    tio.OneHot(num_classes=5)    
    
])

In [ ]:
#The run training method applies the transforms you set and trains a model based on the parameters set here
run_training(
    #input_data_path must be set to the path to the folder containing the subfolders for each training example.
    #Each subfolder should contain one nii.gz file for each of the imaging series and the segmentation for that example
    #The name of each nii.gz file should be the name of the parent folder followed by the name of the imaging series type or seg if it is the segmentation
    #For example,MICCAI_BraTS2020_TrainingData contains ~300 folders, each corresponding to an input example,
    # one folder BraTS20_Training_001, contains five files: BraTS20_Training_001_flair.nii.gz, BraTS20_Training_001_seg.nii.gz, BraTS20_Training_001_t1.nii.gz , BraTS20_Training_001_t2.nii.gz,and BraTS20_Training_001_t1ce.nii.gz
    input_data_path = '../brats_new/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData',
    
    #Where you want your trained model to be saved after training is completed
    output_model_path = './Models/test_train_many_1e-3.pt',
    
    #The transforms you created previously
    training_transform = training_transform,    
    validation_transform = validation_transform,
    
    #The names of the modalities every example in your input data has
    input_channels_list = ['flair','t1','t2','t1ce'],
    
    #Which of the labels in your segmentation you want to train your model to predict
    seg_channels = [1,2,4],
    
    #The name of the type of model you want to train, currently UNet3D is the only available model
    model_type = 'UNet3D',
    
    #The amount of examples per training batch, reduce/increase this based on memory availability
    batch_size = 1,
    
    #The amount of cpus you want to be avaiable for loading the input data into the model
    num_loading_cpus = 1,
    
    #The learning rate of the AdamW optimizer
    learning_rate = 1e-3,
    
    #Whether or not you want to run wandb logging of your run, install wandb to use these parameters
    wandb_logging = False,
    wandb_project_name = None,
    wandb_run_name = None,
    
    #The seed determines how your training and validation data will be randomly split
    #training_split_ratio is the share of your input data you want to use for training the model, the remainder is used for the validation data
    #Keep track of both the seed and ratio used if you want to be able to split your input data the same way in the future
    seed=42,    
    training_split_ratio = 0.9,
    
    #Any parameters which can be applied to a pytorch lightning trainer can also be applied, below is a selection of parameters you can apply
    #Refer to https://pytorch-lightning.readthedocs.io/en/latest/common/trainer.html#trainer-class-api to see the other parameters you could apply
    max_epochs=10,
    amp_backend = 'apex',
    amp_level = 'O1',
    precision=16,
    check_val_every_n_epoch = 1,
    log_every_n_steps=10,      
    val_check_interval= 50,
    progress_bar_refresh_rate=1,      
)

## Evaluation Example

#### If you want to evaluate your model in the future on a certain test dataset follow the below

In [ ]:
#First import the training tool and the torchio library
import sys
sys.path.append('./Radiology_and_AI')
from training.run_training import run_eval
import torchio as tio

In [ ]:
#Whatever normalization and data preperation steps you performed must also be applied here
#Refer to the 
#These transforms are applied to data before it is used to determined the performance of the model on the validation set
test_transform = tio.Compose([
    #Normalization
    tio.ZNormalization(masking_method=tio.ZNormalization.mean),
    
    #Preparation
    tio.CropOrPad((240, 240, 160)),        
    tio.OneHot(num_classes=5)    
    
])

In [ ]:
#The run_eval method evaluates and prints your models performance on a test dataset by averaging the Dice loss per batch
run_eval(
    #The path to the folder containing the data, refer to the training example for more info
    input_data_path= '../brats_new/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData',
    
    #The path to the saved model weights
    model_path="../randgamma.pt",
    
    #The transforms you specified above
    validation_transform=validation_transform,   
    
    #The names of the modalities every example in your input data has
    input_channels_list = ['flair','t1','t2','t1ce'],
    #Which of the labels in your segmentation you want to train your model to predict
    seg_channels = [1,2,4],
    #The name of the type of model you want to train, currently UNet3D is the only available model
    model_type = 'UNet3D'
    
    #If set to true, we only return the performance of the model on the example which were not used for training, based on the train_val_split_ration and seed
    #If false we evaluate on all data and ignore seed and training_split_ratio,
    #set to false if input_data_path is set to a dataset you did not use during training
    is_validation_data = True,
    training_split_ratio=0.9,
    seed=42,
    
    #The amount of examples per training batch, reduce/increase this based on memory availability
    batch_size=1,
    #The amount of cpus you want to be avaiable for loading the input data into the model
    num_loading_cpus = 1,   
)

## Visualization Example